In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [ ]:
import sys
print(sys.executable)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import pickle

import h5py
import re
import gc
import sys

import keras

from keras.models import Model, load_model
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, \
        BatchNormalization, GRU, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences

import time
import datetime

from sklearn.metrics import roc_curve, auc



#  Reading the data

# Creating train/val data

### partition to train/val

In [ ]:
precent_train = 75

In [ ]:
# the seed HAS to be right above the np.random.permutation
np.random.seed(seed=17)
permutated_filenames = list(np.random.permutation(filenames))
train_filenames = permutated_filenames[:round(len(filenames)*precent_train/100.0)]
val_filenames = permutated_filenames[round(len(filenames)*precent_train/100.0):]
train_inds = np.nonzero(df_data['fromFile'].apply(lambda x: x in train_filenames))
val_inds = np.nonzero(df_data['fromFile'].apply(lambda x: x in val_filenames))

### translate df to X,y

In [ ]:
df_data_train = df_data.iloc[train_inds[0]]
df_data_val = df_data.iloc[val_inds[0]]


# Preprocessing before LSTM

In [ ]:
feature_sequences.iloc[0].shape

In [ ]:
y_feature_sequences = pd.get_dummies(concat_temp.type)

In [ ]:
# input_sequences1 = Input(shape=(10,50,))

# gru_1 = GRU(units=16, return_sequences=True)(input_sequences1)
# gru_2 = GRU(units=16, return_sequences=True)(gru_1)
# reshape_1 = Reshape(target_shape=(10,16,1))(gru_2)

# pool1 = MaxPooling2D(pool_size=(10,4), strides=(1,4))(reshape_1)
# pool_gru = Reshape(target_shape=(1,4))(pool1)

# flatten_gru = Flatten()(pool_gru)

# output_gru = Dense(4, activation='softmax')(flatten_gru)
# gru_model = Model(inputs=[input_sequences1], outputs=[output_gru])
# gru_model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
# gru_model.summary()

In [ ]:
input_sequences1 = Input(shape=(20,120,))

gru_1 = GRU(units=100, return_sequences=True)(input_sequences1)
gru_2 = GRU(units=30)(gru_1)

dropout_gru1 = Dropout(0.3)(gru_2)

dense_gru = Dense(20, activation='tanh')(dropout_gru1)

output_gru = Dense(4, activation='softmax')(dense_gru)
gru_model = Model(inputs=[input_sequences1], outputs=[output_gru])
gru_model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
gru_model.summary()

In [ ]:
early_cb = EarlyStopping(monitor='val_loss', patience=35, verbose=0, mode='auto')

In [ ]:
# gru_model.fit(np.swapaxes(np.array(list(feature_sequences)), 1, 2), y_feature_sequences.values, \
#             epochs=1000, validation_split=0.2, batch_size=20, callbacks=[early_cb])

In [ ]:
gru_model.fit(np.array(list(feature_sequences)), y_feature_sequences.values, \
            epochs=1000, validation_split=0.2, batch_size=50, callbacks=[early_cb])

In [ ]:
np.array(list(feature_sequences)).shape

# End of GRU code